Benchmarking is essential for understanding exactly how well your DeepSeek reasoning agent performs compared to traditional AI models—and where you can make improvements. In this chapter, you'll learn to create comprehensive accuracy tests, run systematic comparisons, and generate clear reports that show your agent's strengths and areas for improvement.

**Creating Your Test Suite**

The foundation of effective benchmarking is a well-designed test suite that covers various types of reasoning challenges. You'll create a collection of test cases that represent the kinds of problems your agent will encounter in real-world use.

Start by choosing a variety of reasoning challenges. You'll want to include different types of problems such as:

·       Mathematical calculations and word problems

·       Logical deductions and reasoning chains

·       Code analysis and debugging scenarios

·       Reading comprehension and inference tasks

·       Creative problem-solving challenges

**Documenting Your Test Cases**

You'll store your test cases in a structured format that's easy for your code to process. Create a JSON file that clearly defines each test case with its input, expected output, and category.

Here's how to structure your test cases:
\[  
   {"type": "math", "input": "25% of 80?", "expected": "20"},  
   {"type": "logic", "input": "All cats are mammals. Fluffy is a cat. Is Fluffy a mammal?", "expected": "Yes"},  
   {"type": "code", "input": "Find the bug: for i in range(10) print(i)", "expected": "Missing colon after range(10)"},  
   {"type": "reasoning", "input": "If it rains, the ground gets wet. The ground is wet. Did it rain?", "expected": "Cannot determine \- other things can make ground wet"}  
 \]

When you create this file, save it as test\_cases.json in your project directory. You'll be able to see this file in your file explorer, and your Python code will read from it during testing.

For larger test suites, you might prefer a CSV format which you can easily edit in a spreadsheet application:

type,input,expected  
 math,"25% of 80?","20"  
 logic,"All cats are mammals. Fluffy is a cat. Is Fluffy a mammal?","Yes"  
 code,"Find the bug: for i in range(10) print(i)","Missing colon after range(10)"

**Building Your Testing Framework**

Now you'll create a Python script that systematically runs your test cases through different models and collects the results for comparison.

**Setting Up the Test Runner**

Create a function that loads your test cases and runs them through any model configuration you specify:


In [ ]:
import json  
 import time

 def load\_test\_cases(filename):  
 	"""Load test cases from a JSON file"""  
 	with open(filename, 'r') as file:  
     	return json.load(file)

 def run\_test(model\_config, test\_cases):  
 	"""Run all test cases through a specific model configuration"""  
 	results \= \[\]  
 	  
 	for case in test\_cases:  
     	print(f"Running test: {case\['input'\]\[:50\]}...")  \# Show progress  
     	  
     	\# Record start time  
     	start\_time \= time.time()  
     	  
     	\# Create and execute the task  
     	task \= Task(case\["input"\], agent=Agent(llm\_config=model\_config))  
     	response \= task.execute()  
     	  
     	\# Calculate response time  
     	response\_time \= time.time() \- start\_time  
     	  
     	\# Store the complete result  
     	results.append({  
         	"input": case\["input"\],  
         	"response": response.content,  
         	"expected": case\["expected"\],  
         	"type": case\["type"\],  
         	"response\_time": response\_time  
     	})  
     	  
     	print(f"  Completed in {response\_time:.2f} seconds")  
 	  
 	return results


When you run this function, you'll see progress messages on your screen showing which test is currently running and how long each one takes to complete.

**Implementing Response Scoring**

You'll need a systematic way to evaluate whether each response is correct. Create a scoring function that compares actual responses to expected answers:


In [ ]:
def score\_response(response, expected, test\_type):  
 	"""Score a response against the expected answer"""  
 	response\_lower \= response.lower().strip()  
 	expected\_lower \= expected.lower().strip()  
 	  
 	\# Basic correctness check  
 	correct \= expected\_lower in response\_lower  
 	  
 	\# You can add more sophisticated scoring logic here  
 	if test\_type \== "math":  
     	\# For math problems, look for exact number matches  
     	import re  
     	expected\_numbers \= re.findall(r'\\d+\\.?\\d\*', expected)  
 	    response\_numbers \= re.findall(r'\\d+\\.?\\d\*', response)  
     	correct \= any(num in response\_numbers for num in expected\_numbers)  
 	  
 	elif test\_type \== "logic":  
     	\# For logic problems, check for yes/no or true/false  
     	if "yes" in expected\_lower or "true" in expected\_lower:  
         	correct \= ("yes" in response\_lower or "true" in response\_lower)  
     	elif "no" in expected\_lower or "false" in expected\_lower:  
         	correct \= ("no" in response\_lower or "false" in response\_lower)  
 	  
 	return {  
     	"correct": correct,  
     	"confidence": 1.0 if correct else 0.0,  \# You can make this more nuanced  
     	"response\_length": len(response)  
 	}


This function will return a dictionary with scoring information for each test case, which you'll use in your analysis.

**Running Comparative Tests**

Now you'll run your test suite through multiple models to compare their performance. You'll test both a baseline model and your DeepSeek reasoning agent.

**Setting Up Model Configurations**

First, define the configurations for the models you want to compare:


In [ ]:
\# Baseline model configuration  
 baseline\_config \= {  
 	"model": "gpt-3.5-turbo",  
 	"temperature": 0.1,  
 	"max\_tokens": 500  
 }

 \# DeepSeek reasoning agent configuration   
 reasoning\_config \= {  
 	"model": "deepseek-reasoner",  
 	"temperature": 0.1,  
 	"max\_tokens": 1000,  
 	"reasoning\_mode": True  
 }

**Executing the Comparison**

Load your test cases and run them through both models:


In [ ]:
\# Load your test cases  
 test\_cases \= load\_test\_cases("test\_cases.json")  
 print(f"Loaded {len(test\_cases)} test cases")

 \# Run baseline tests  
 print("\\nRunning baseline model tests...")  
 baseline\_results \= run\_test(baseline\_config, test\_cases)

 \# Run reasoning agent tests  
 print("\\nRunning reasoning agent tests...")  
 reasoning\_results \= run\_test(reasoning\_config, test\_cases)

 print("\\nAll tests completed\!")


As this runs, you'll see progress messages telling you which model is being tested and which specific test case is currently running.

**Analyzing Your Results**

Once you have results from both models, you'll analyze them statistically to understand the performance differences.

**Calculating Basic Statistics**

Use Python's built-in statistics module to compute accuracy metrics:


In [ ]:
import statistics

 def analyze\_results(results, model\_name):  
 	"""Analyze test results and return key metrics"""  
 	  
 	\# Score all responses  
 	scores \= \[\]  
 	response\_times \= \[\]  
 	type\_performance \= {}  
 	  
 	for result in results:  
     	score \= score\_response(result\["response"\], result\["expected"\], result\["type"\])  
         scores.append(score\["correct"\])  
         response\_times.append(result\["response\_time"\])  
     	  
     	\# Track performance by test type  
     	test\_type \= result\["type"\]  
         if test\_type not in type\_performance:  
         	type\_performance\[test\_type\] \= \[\]  
         type\_performance\[test\_type\].append(score\["correct"\])  
 	  
 	\# Calculate overall metrics  
 	accuracy \= statistics.mean(scores)  
 	avg\_response\_time \= statistics.mean(response\_times)  
 	  
 	\# Calculate performance by type  
 	type\_accuracies \= {}  
 	for test\_type, type\_scores in type\_performance.items():  
     	type\_accuracies\[test\_type\] \= statistics.mean(type\_scores)  
 	  
 	return {  
     	"model": model\_name,  
     	"overall\_accuracy": accuracy,  
     	"avg\_response\_time": avg\_response\_time,  
     	"total\_tests": len(results),  
     	"correct\_answers": sum(scores),  
     	"type\_accuracies": type\_accuracies  
 	}

 \# Analyze both sets of results  
 baseline\_analysis \= analyze\_results(baseline\_results, "Baseline Model")  
 reasoning\_analysis \= analyze\_results(reasoning\_results, "DeepSeek Reasoning Agent")

 \# Calculate improvement  
 accuracy\_improvement \= reasoning\_analysis\["overall\_accuracy"\] \- baseline\_analysis\["overall\_accuracy"\]  
 time\_difference \= reasoning\_analysis\["avg\_response\_time"\] \- baseline\_analysis\["avg\_response\_time"\]

 print(f"\\nResults Summary:")  
 print(f"Baseline Accuracy: {baseline\_analysis\['overall\_accuracy'\]:.1%}")  
 print(f"Reasoning Agent Accuracy: {reasoning\_analysis\['overall\_accuracy'\]:.1%}")  
 print(f"Improvement: {accuracy\_improvement:.1%}")


When you run this analysis, you'll see a clear summary printed to your screen showing the key performance differences between the models.

**Creating Visual Reports**

Visual reports make it much easier to understand and communicate your benchmarking results. You'll create charts and tables that clearly show performance differences.

**Setting Up Data for Visualization**

First, organize your data into a format that's easy to visualize:


In [ ]:
import pandas as pd  
 import matplotlib.pyplot as plt

 def create\_comparison\_dataframe(baseline\_analysis, reasoning\_analysis):  
 	"""Create a DataFrame for easy visualization"""  
 	  
 	\# Overall comparison  
 	overall\_data \= {  
     	"Model": \["Baseline", "Reasoning Agent"\],  
     	"Accuracy": \[baseline\_analysis\["overall\_accuracy"\], reasoning\_analysis\["overall\_accuracy"\]\],  
     	"Avg Response Time": \[baseline\_analysis\["avg\_response\_time"\], reasoning\_analysis\["avg\_response\_time"\]\]  
 	}  
 	  
 	overall\_df \= pd.DataFrame(overall\_data)  
 	  
 	\# Performance by type  
 	all\_types \= set(baseline\_analysis\["type\_accuracies"\].keys()) | set(reasoning\_analysis\["type\_accuracies"\].keys())  
 	type\_data \= \[\]  
 	  
 	for test\_type in all\_types:  
     	baseline\_acc \= baseline\_analysis\["type\_accuracies"\].get(test\_type, 0\)  
     	reasoning\_acc \= reasoning\_analysis\["type\_accuracies"\].get(test\_type, 0\)  
     	  
     	type\_data.append({  
         	"Test Type": test\_type,  
         	"Baseline": baseline\_acc,  
         	"Reasoning Agent": reasoning\_acc,  
         	"Improvement": reasoning\_acc \- baseline\_acc  
     	})  
 	  
 	type\_df \= pd.DataFrame(type\_data)  
 	  
 	return overall\_df, type\_df

 \# Create the dataframes  
 overall\_df, type\_df \= create\_comparison\_dataframe(baseline\_analysis, reasoning\_analysis)


**Generating Charts**

Create clear, informative charts that highlight your findings:


In [ ]:
\# Create accuracy comparison chart  
 fig, (ax1, ax2) \= plt.subplots(1, 2, figsize=(15, 6))

 \# Overall accuracy comparison  
 overall\_df.plot.bar(x="Model", y="Accuracy", ax=ax1, color=\['lightblue', 'darkblue'\])  
 ax1.set\_title("Overall Accuracy Comparison")  
 ax1.set\_ylabel("Accuracy Rate")  
 ax1.set\_ylim(0, 1\)  
 ax1.tick\_params(axis='x', rotation=0)

 \# Performance by test type  
 type\_df.plot.bar(x="Test Type", y=\["Baseline", "Reasoning Agent"\], ax=ax2)  
 ax2.set\_title("Accuracy by Test Type")  
 ax2.set\_ylabel("Accuracy Rate")  
 ax2.set\_ylim(0, 1\)  
 ax2.tick\_params(axis='x', rotation=45)

 plt.tight\_layout()  
 plt.savefig("accuracy\_comparison.png", dpi=300, bbox\_inches='tight')  
 plt.show()


When you run this code, you'll see two bar charts appear on your screen. The first shows overall accuracy comparison, and the second breaks down performance by test type. You'll also find a high-quality PNG file saved in your project directory.

**Creating a Detailed Report Table**

Generate a comprehensive table that summarizes all your findings:


In [ ]:
def generate\_detailed\_report(baseline\_analysis, reasoning\_analysis):  
 	"""Generate a detailed performance report"""  
 	  
 	print("=" \* 60\)  
 	print("BENCHMARK ACCURACY COMPARISON REPORT")  
 	print("=" \* 60\)  
 	  
 	print(f"\\nOVERALL PERFORMANCE:")  
 	print(f"{'Metric':\<25} {'Baseline':\<15} {'Reasoning Agent':\<15} {'Change':\<10}")  
 	print("-" \* 70\)  
 	  
 	\# Overall accuracy  
 	acc\_change \= reasoning\_analysis\["overall\_accuracy"\] \- baseline\_analysis\["overall\_accuracy"\]  
 	print(f"{'Accuracy':\<25} {baseline\_analysis\['overall\_accuracy'\]:\<15.1%} {reasoning\_analysis\['overall\_accuracy'\]:\<15.1%} {acc\_change:+.1%}")  
 	  
 	\# Response time  
 	time\_change \= reasoning\_analysis\["avg\_response\_time"\] \- baseline\_analysis\["avg\_response\_time"\]  
 	print(f"{'Avg Response Time (s)':\<25} {baseline\_analysis\['avg\_response\_time'\]:\<15.2f} {reasoning\_analysis\['avg\_response\_time'\]:\<15.2f} {time\_change:+.2f}")  
 	  
 	\# Total tests  
 	print(f"{'Total Test Cases':\<25} {baseline\_analysis\['total\_tests'\]:\<15} {reasoning\_analysis\['total\_tests'\]:\<15} {'--':\<10}")  
 	  
 	print(f"\\nPERFORMANCE BY TEST TYPE:")  
 	print(f"{'Test Type':\<15} {'Baseline':\<15} {'Reasoning Agent':\<15} {'Improvement':\<12}")  
 	print("-" \* 60\)  
 	  
 	for test\_type in baseline\_analysis\["type\_accuracies"\]:  
     	baseline\_acc \= baseline\_analysis\["type\_accuracies"\]\[test\_type\]  
     	reasoning\_acc \= reasoning\_analysis\["type\_accuracies"\].get(test\_type, 0\)  
     	improvement \= reasoning\_acc \- baseline\_acc  
     	  
     	print(f"{test\_type:\<15} {baseline\_acc:\<15.1%} {reasoning\_acc:\<15.1%} {improvement:+.1%}")  
 	  
 	print("=" \* 60\)

\# Generate the detailed report  
 generate\_detailed\_report(baseline\_analysis, reasoning\_analysis)



This will print a comprehensive report to your screen that you can copy and save for documentation or sharing with others.

**Interpreting and Summarizing Your Findings**

After running your analysis, you'll want to create clear, actionable summaries of what you discovered.

**Writing Your Summary**

Based on your results, write a clear summary that highlights the key findings:


In [ ]:
def create\_executive\_summary(baseline\_analysis, reasoning\_analysis):  
 	"""Create an executive summary of benchmarking results"""  
 	  
 	accuracy\_improvement \= reasoning\_analysis\["overall\_accuracy"\] \- baseline\_analysis\["overall\_accuracy"\]  
 	time\_impact \= reasoning\_analysis\["avg\_response\_time"\] \- baseline\_analysis\["avg\_response\_time"\]  
 	  
 	summary \= f"""  
 EXECUTIVE SUMMARY:  
 The DeepSeek reasoning agent demonstrated significant performance improvements over the baseline model:

 • Overall accuracy increased from {baseline\_analysis\['overall\_accuracy'\]:.1%} to {reasoning\_analysis\['overall\_accuracy'\]:.1%}  
   (improvement of {accuracy\_improvement:+.1%})

 • Average response time changed by {time\_impact:+.2f} seconds per query

 • Tested across {reasoning\_analysis\['total\_tests'\]} diverse test cases covering multiple reasoning types

 KEY INSIGHTS:  
 """  
 	  
 	\# Find best and worst performing categories  
 	improvements\_by\_type \= {}  
 	for test\_type in baseline\_analysis\["type\_accuracies"\]:  
     	baseline\_acc \= baseline\_analysis\["type\_accuracies"\]\[test\_type\]  
     	reasoning\_acc \= reasoning\_analysis\["type\_accuracies"\].get(test\_type, 0\)  
     	improvements\_by\_type\[test\_type\] \= reasoning\_acc \- baseline\_acc  
 	  
 	best\_category \= max(improvements\_by\_type.keys(), key=lambda k: improvements\_by\_type\[k\])  
 	worst\_category \= min(improvements\_by\_type.keys(), key=lambda k: improvements\_by\_type\[k\])  
 	  
 	summary \+= f"""  
 • Strongest improvement in {best\_category} tasks ({improvements\_by\_type\[best\_category\]:+.1%})  
 • Areas for improvement: {worst\_category} tasks ({improvements\_by\_type\[worst\_category\]:+.1%})

 RECOMMENDATIONS:  
 • Deploy reasoning agent for production use \- shows clear accuracy benefits  
 • Monitor response time impact on user experience  
 • Consider additional training data for {worst\_category} tasks  
 • Implement continuous benchmarking to track future improvements  
 """  
 	  
 	return summary

 \# Generate and display the executive summary  
 summary \= create\_executive\_summary(baseline\_analysis, reasoning\_analysis)  
 print(summary)


**Setting Up Continuous Benchmarking**

Benchmarking shouldn't be a one-time activity. You'll want to integrate continuous benchmarking into your development workflow to track improvements over time and make data-driven decisions about agent updates.

**Creating an Automated Benchmark Runner**

Set up a system that can run benchmarks automatically on a schedule:


In [ ]:
import datetime  
 import os

 def run\_continuous\_benchmark():  
 	"""Run a complete benchmark and save results with timestamp"""  
 	  
 	timestamp \= datetime.datetime.now().strftime("%Y-%m-%d\_%H-%M-%S")  
 	results\_dir \= f"benchmark\_results\_{timestamp}"  
 	os.makedirs(results\_dir, exist\_ok=True)  
 	  
 	print(f"Starting benchmark run: {timestamp}")  
 	  
 	\# Load test cases  
 	test\_cases \= load\_test\_cases("test\_cases.json")  
 	  
 	\# Run tests  
 	baseline\_results \= run\_test(baseline\_config, test\_cases)  
 	reasoning\_results \= run\_test(reasoning\_config, test\_cases)  
 	  
 	\# Analyze results  
 	baseline\_analysis \= analyze\_results(baseline\_results, "Baseline")  
 	reasoning\_analysis \= analyze\_results(reasoning\_results, "Reasoning Agent")  
 	  
 	\# Save detailed results  
 	results\_summary \= {  
     	"timestamp": timestamp,  
     	"baseline\_analysis": baseline\_analysis,  
     	"reasoning\_analysis": reasoning\_analysis,  
     	"test\_cases\_count": len(test\_cases)  
 	}  
 	  
 	\# Save to JSON file  
 	with open(f"{results\_dir}/benchmark\_results.json", "w") as f:  
     	json.dump(results\_summary, f, indent=2)  
 	  
 	\# Generate and save report  
 	summary \= create\_executive\_summary(baseline\_analysis, reasoning\_analysis)  
 	with open(f"{results\_dir}/executive\_summary.txt", "w") as f:  
     	f.write(summary)  
 	  
 	print(f"Benchmark completed. Results saved in {results\_dir}/")  
 	  
 	return results\_summary

 \# Run a benchmark  
 benchmark\_results \= run\_continuous\_benchmark()


When you run this function, you'll see a new directory created with a timestamp in its name, containing all your benchmark results and reports.

**Tracking Performance Over Time**

Keep a history of your benchmark results to identify trends and improvements:


In [ ]:
def compare\_historical\_results(results\_dir="benchmark\_history"):  
 	"""Compare current results with historical benchmarks"""  
 	  
 	if not os.path.exists(results\_dir):  
     	print("No historical results found. This will be your baseline.")  
     	return  
 	  
 	\# Load all historical results  
 	historical\_files \= \[f for f in os.listdir(results\_dir) if f.endswith("benchmark\_results.json")\]  
 	  
 	if not historical\_files:  
     	print("No previous benchmark files found.")  
     	return  
 	  
 	\# Load the most recent previous result  
     historical\_files.sort(reverse=True)  \# Most recent first  
 	latest\_historical \= historical\_files\[0\]  
 	  
 	with open(os.path.join(results\_dir, latest\_historical), 'r') as f:  
     	previous\_results \= json.load(f)  
 	  
 	print(f"Comparing with previous benchmark from: {previous\_results\['timestamp'\]}")  
 	print(f"Previous accuracy: {previous\_results\['reasoning\_analysis'\]\['overall\_accuracy'\]:.1%}")  
 	  
 	\# You can expand this to show trends over multiple benchmark runs


**Moving Forward with Your Benchmarking Strategy**

You now have a complete benchmarking system that can measure your DeepSeek reasoning agent's performance, compare it to baseline models, and track improvements over time.

Your benchmarking framework includes:

·       A structured test suite covering multiple reasoning types

·       Automated testing and scoring systems

·       Statistical analysis and visual reporting

·       Continuous monitoring capabilities

Remember to regularly update your test cases to reflect new use cases and challenges your agent will face. As you make improvements to your agent, run benchmarks to verify that changes actually improve performance rather than just changing it.

In upcoming chapters, we'll use these benchmarking insights to guide optimization strategies and ensure your reasoning agent continues to improve in real-world applications.

